# Python Decorators / Functional Programming
***from https://realpython.com/primer-on-python-decorators/***

Python `decorators` provide simple syntax for `higher-order functions`

`Higher order functions` do on of two things
 - take one or more functions as arguments (procedural parameters)
 - returns a functions as its result


In [1]:
def twice(f):
    def result(a):
        return f(f(a))
    return result

plusthree = lambda x : x + 3
g = twice(plusthree)
g(7)

13

In [2]:
# with Python Decorators
@twice
def g(x):
    return x + 3

g(7)

13

-----

### Functions
 - return a value based on given arguments
 - may have side effects

e.g. print( ) functions return None and dislpay on console

In [5]:
print(g(7))

13


- ***first class objects*** can be passed around and used as arguments

In Python, `functions`are ***first class objects*** like `str`, `int`, `list`, etc

In [7]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

`greet_bob` is a ***higher order function*** 

In [8]:
greet_bob(say_hello)

'Hello Bob'

In [25]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

---------

### Inner Functions
 - defining your function inside another function 

In [12]:
def parent():
    print("Printing from the parent() function")
    
    def first_child():
        print("Printing from the first_child() function")
        
    def second_child():
        print("Printing from the second_child() function")
    
    second_child()
    first_child()

calling the `parent()` function will call both the children functions

In [14]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


but calling one of the children functions throws an `error`
<br> They are locally `scoped` to the `parent()`

In [26]:
first_child()

NameError: name 'first_child' is not defined

-------

### Returning Funtions from Functions

In [16]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"
    
    def second_child():
        return "Call me Liam"
    
    if num == 1:
        return first_child
    else:
        return second_child

*note when you are using a function with/without parenthesis*
<br> That means you are returning a ***reference*** to the function and not ***calling*** it.

In [20]:
first = parent(1)
second = parent(2)

In [21]:
first

<function __main__.parent.<locals>.first_child()>

In [22]:
second

<function __main__.parent.<locals>.second_child()>

because you have the ***reference*** to the function assigned to a variable
<br> you can call the function by adding `()` to the variable name

In [23]:
first()

'Hi, I am Emma'

In [33]:
second()

'Call me Liam'

------

### Simple Decorators

In [43]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper  # This does not call the wrapper function... it references it.

def say_whee():
    print("Whee!")
    
say_whee = my_decorator(say_whee)  # This is where "decoration" happens

This is variable that contains a function `my_decorator` 
<br>that returns a reference to its child function `wrapper`
<br>that takes a function `say_whee` as an argument and calls it
<br> This variable `say_whee` points to the `wrapper()` inner function

In [42]:
say_whee  

<function __main__.my_decorator.<locals>.wrapper()>

In [40]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


***Decorators wrap a function and modifies its behavior***
<br> So instead of just printing `Whee!` the `wrapper()`can do something before and/or after the function call.

In [45]:
from datetime import datetime

In [54]:
def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour <22:
            func()
            print("Let's go out and play")
        else:
            print("SHH...the neighbors are asleep")
            pass  
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)  # Again, this is where decoration happens

In [55]:
say_whee()

Whee!
Let's go out and play


-----

### Syntactic Sugar
Let's make this easier to read with some syntactic sugar

In [56]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator  # This is an easier way of saying `say_whee = ....`
def say_whee():
    print("Whee!")

In [58]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


--------

### Reusing Decorators
Create a file called `decorators.py` with the following content:
 
```
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice
```


In [1]:
from decorators import do_twice

In [3]:
@do_twice
def say_whee():
    print("Whee!")

In [81]:
say_whee()

Whee!
Whee!


--------

### Decorating Functions with Arguments

In [2]:
@do_twice
def greet(name):
    print(f"Hello {name}")

In [70]:
greet("World")

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

This `error` is thrown because we have placed an argument inside the `()`.
<br> You will notice that `say_whee()` does not have any arguments.

<br> So we can see that the function we are defining is really pointing to the function in the decorator.
<br> If the function in the decorator does not take more than one argument then our decorated function canot either.
<br> We must modify the `decorators.py` file to permit more than one arguement to be used:
```
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice
```

In [88]:
say_whee()

Whee!
Whee!


In [3]:
greet("World")

Hello World
Hello World


-------

### Returning Values from Decorated Functions

In [1]:
from decorators import do_twice

In [2]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [8]:
hi_adam = return_greeting("Adam")

Creating greeting
Creating greeting


In [9]:
print(hi_adam)

None


The decorator ate the return value.
<br> The `wrapper_do_twice()` does not return a value but a reference to a function.
<br> The `decorators.py` file need to be modified:
```
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice
```

In [10]:
from decorators import do_twice

In [3]:
return_greeting("Adam")

Creating greeting
Creating greeting


'Hi Adam'

In [4]:
print

<function print>

In [5]:
print.__name__

'print'

In [6]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [9]:
say_whee

<function decorators.do_twice.<locals>.wrapper_do_twice(*args, **kwargs)>

In [10]:
say_whee.__name__

'wrapper_do_twice'

In [11]:
help(say_whee)

Help on function wrapper_do_twice in module decorators:

wrapper_do_twice(*args, **kwargs)



We can see that `say_whee` has got some complications inside a `wrapper_do_twice` and decorator `do_twice`
<br> We need to modify the `decorators.py` again:
```
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice
```
The `@functools.wraps(func)` is applied to wrapper function of a decorator.
<br>It updates the `wrapper` function to look like the `wrapped` function which makes it easier to identify.

In [1]:
from decorators import do_twice

In [4]:
say_whee

<function __main__.say_whee()>

In [5]:
say_whee.__name__

'say_whee'

In [6]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



-----

### Timing Functions

In [12]:
import functools
import time

In [21]:
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10_000)])

In [24]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0045 secs


In [25]:
waste_some_time(999)

Finished 'waste_some_time' in 3.0573 secs


`@timer` decorator is fine for general timing but a more precise timer is `timeit` module

-------

### Debugging Code

In [14]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")
        return value
    return wrapper_debug

In [38]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [39]:
make_greeting("Benjamin")

Calling make_greeting('Benjamin')
'make_greeting' returned 'Howdy Benjamin!'


'Howdy Benjamin!'

In [40]:
make_greeting("Richard", age = 112)

Calling make_greeting('Richard', age=112)
'make_greeting' returned 'Whoa Richard! 112 already, you are growing up!'


'Whoa Richard! 112 already, you are growing up!'

In [41]:
make_greeting(name="Dorrisile", age=116)

Calling make_greeting(name='Dorrisile', age=116)
'make_greeting' returned 'Whoa Dorrisile! 116 already, you are growing up!'


'Whoa Dorrisile! 116 already, you are growing up!'

***A better example*** : Math functions

In [1]:
import math 
from decorators import debug

# Apply a decoraotor to a standard library function
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

In [2]:
approximate_e(5)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24


2.708333333333333

-----

### Slowing Down Code
 - you might want to rate-limit a function
 - you might want text to on screen more slowly give the user a more paced experience

In [4]:
import functools
import time

In [20]:
def slow_down(func):
    """Sleep 1 second defore calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number <1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [21]:
countdown(3)

3
2
1
Liftoff!


This `countdown` function is a recursive function which means it is a function that calls itself

----

### Registering Plugins
Decorators don't have to wrap the function they're decorating. They can also simply register that a function exists and return it unwrapped. This can be used to create a lightweight plug-in architecture

In [1]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using '{greeter}'")
    return greeter_func(name)

In [4]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

In [5]:
randomly_greet("Alice")

Using 'say_hello'


'Hello Alice'

In [2]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import random\nPLUGINS = dict()\n\ndef register(func):\n    """Register a function as a plug-in"""\n    PLUGINS[func.__name__] = func\n    return func\n\n@register\ndef say_hello(name):\n    return f"Hello {name}"\n\n@register\ndef be_awesome(name):\n    return f"Yo {name}, together we are the awesomest!"\n\ndef randomly_greet(name):\n    greeter, greeter_func = random.choice(list(PLUGINS.items()))\n    print(f"Using \'{greeter}\'")\n    return greeter_func(name)',
  'globals()'],
 '_oh': {},
 '_dh': ['/Users/claytonlouden/jup'],
 'In': ['',
  'import random\nPLUGINS = dict()\n\ndef register(func):\n    """Register a function as a plug-in"""\n    PLUGINS[func.__name__] = func\n    return func\n\n@register\ndef 

`global()` is similar but with `@register` you can curate your own list of variables.

-----

### Is user Logged in?

In [5]:
from flask import Flask, g, request, redirect, url_for
import functools
app = Flask(__name__)

In [4]:
def login_required(func):
    """Make sure user is logged in before proceeding"""
    @functools.wraps(func)

ModuleNotFoundError: No module named 'Flask'